<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/query-expansion/query_expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import collections
import matplotlib.pyplot as plt
%matplotlib inline

import heapq as hp

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Introdução

Nesta atividade exercitaremos conceitos sobre expansão de consultas.  Os dados utilizados são de textos de notícias resultados da utilização de técnicas de crawling e scrapping. Os scripts para a coleta desses dados podem ser encontrados [neste](https://github.com/valterlucena/ri_lab_01) repositório.

Primeiramente, vamos importar os dados que serão utilizados.


In [0]:
DATA_URL = 'https://raw.githubusercontent.com/valterlucena/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL).replace(np.nan, '', regex=True)

Agora iremos criar nosso índice invertido, para ser utilizado mais adiante. Utilizaremos a função tokenize da biblioteca NLTK associada à uma expressão regular, que considerará como token apenas as sequências de caracteres não-especiais (com exceção do hífen) ou numéricos que não formem stopwords.

In [0]:
toker = RegexpTokenizer('''\w+[-']*\w*''')
stop_words = stopwords.words('portuguese')

def isValid(token):
  return token not in stop_words and not bool(re.search(r'\d', token))

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = [token for token in toker.tokenize(document.lower()) if isValid(token)]
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

In [0]:
index = build_index(news.text)

In [0]:
# numero de documentos que contém cada termo
def count_docs(termo):
  return len(index[termo].keys())

def count_docs_conj(a, b):
  docs_a = index[a].keys()
  docs_b = index[b].keys()
  count = 0
  for doc in docs_a:
    if doc in docs_b:
      count += 1
  return count  

def total_docs():
  return news.text.count()

In [0]:
# mutual information (MIM)
def mutual_information(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  mim = n_ab / (n_a * n_b)
  return mim

# expect mutual information (EMIM)
def expected_mutual_information(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  n = total_docs()
  exp = n * (n_ab / (n_a * n_b))
  emim = n_ab * np.log(exp) if exp != 0 else 0
  return emim

# chi-square
def chi_square(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  n = total_docs()
  chi = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return chi

# dice's coefficient
def dice_coefficient(a, b):
  n_a = count_docs(a)
  n_b = count_docs(b)
  n_ab = count_docs_conj(a, b)
  dice = n_ab / (n_a + n_b)
  return dice

In [0]:
def build_metric_list(term, metric):
  pairs = []
  for posting in index.keys():
    if posting != termo:
      if metric == 'mim':
        metric_value = mutual_information(termo, posting)
      elif metric == 'emim':
        metric_value = expected_mutual_information(termo, posting)
      elif metric == 'chi-square':
        metric_value = chi_square(termo, posting)
      elif metric == 'dice':
        metric_value = dice_coefficient(termo, posting)
      pairs.append((posting, metric_value))      
  pairs = sorted(pairs, key = lambda x: x[1], reverse=True)
  return pairs

termos = ['presidente', 'lula', 'bolsonaro', 'governo', 'federal']
termos_metricas = {}
for termo in termos:
  metricas = {}
  metricas['mim'] = build_metric_list(termo, 'mim')
  metricas['emim'] = build_metric_list(termo, 'emim')
  metricas['chi-square'] = build_metric_list(termo, 'chi-square')
  metricas['dice'] = build_metric_list(termo, 'dice')
  termos_metricas[termo] = metricas

In [0]:
def build_table(termo):
  mim = [k for k,_ in termos_metricas[termo]['mim'][0:10]]
  emim = [k for k,_ in termos_metricas[termo]['emim'][0:10]]
  chi_square = [k for k,_ in termos_metricas[termo]['chi-square'][0:10]]
  dice = [k for k,_ in termos_metricas[termo]['dice'][0:10]]
  data = {'MIM': mim, 'EMIM': emim, 'X²': chi_square, 'Dice': dice}
  return pd.DataFrame(data)

In [9]:
build_table(termos[0])

,MIM,EMIM,X²,Dice
0,requereu,bolsonaro,jair,bolsonaro
1,penal,jair,bolsonaro,brasil
2,especial,brasil,brasil,jair
3,embu,ser,direitos,ser
4,artes,governo,processo,governo
5,nando,direitos,apenas,é
6,moura,país,candidato,país
7,injúria,contra,gente,direitos
8,difamação,apenas,antes,contra
9,requerida,dia,ser,dia


In [10]:
build_table(termos[1])

,MIM,EMIM,X²,Dice
0,elio,ex-presidente,luiz,ex-presidente
1,gaspari,dizer,ex-presidente,dizer
2,resolve,democracia,dizer,democracia
3,pedirá,pt,inocência,pt
4,inocência,luiz,inácio,luiz
5,negocio,prisão,legislação,legislação
6,trocaria,direitos,pt,ato
7,relativo,silva,democracia,silva
8,conforto,dentro,curitiba,dentro
9,assemelhariam,legislação,visão,prisão


In [11]:
build_table(termos[2])

,MIM,EMIM,X²,Dice
0,requereu,jair,jair,jair
1,especial,presidente,presidente,presidente
2,embu,ser,onde,ser
3,artes,governo,governo,governo
4,sp,brasil,vai,brasil
5,nando,onde,ser,é
6,moura,país,antes,país
7,injúria,é,brasil,onde
8,difamação,vai,país,sobre
9,requerida,antes,deputados,vai


In [12]:
build_table(termos[3])

,MIM,EMIM,X²,Dice
0,requereu,bolsonaro,reforma,bolsonaro
1,penal,ser,previdência,ser
2,corre,brasil,outra,brasil
3,especial,presidente,justiça,presidente
4,embu,reforma,tudo,é
5,artes,todos,política,todos
6,nando,política,bolsonaro,política
7,moura,federal,falta,país
8,injúria,justiça,brasil,contra
9,difamação,contra,federal,federal


In [13]:
build_table(termos[4])

,MIM,EMIM,X²,Dice
0,defendia,ministro,ministro,ministro
1,lobista,justiça,justiça,justiça
2,denunciou,direitos,tribunal,direitos
3,furnas,governo,prisão,governo
4,dino,prisão,direitos,prisão
5,miraglia,ser,fernando,ano
6,comeu,ano,polícia,nacional
7,pão,nacional,humanos,ser
8,amaçou,público,investigações,ter
9,helicóptero,ter,desde,público
